In [108]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
from functions import *
from helpers import *
from proj1_helpers import *
from costs import *
from data_preprocessing import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [109]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
# TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print("loading of the data : done")

loading of the data : done


## Data preprocessing

In [110]:
# reduction of the size of the training data set: 
# random selection of the data
num_samples=2000
seed=3
y, tX = select_random(y, tX, num_samples, seed)
print ("random selection of samples : done")
print( "number of samples : ", y.shape[0])

##### Splitting the data
xtrain1,xvalid1,ytrain,yvalid=split_data(tX, y, 0.7)
print( "splitting of the data : done ")
print("number of training samples :", xtrain1.shape[0])

#### Cleaning the data
#xtrain1bis=clean_data(xtrain1)
#xvalid1bis=clean_data(xvalid1)
#remove_threshold=0
#bad=clean_remove_data(xtrain1,remove_threshold)
#print("bad columns indices :", bad)
#xtrain1bis=np.delete(xtrain1,bad,axis=1)
#xvalid1bis=np.delete(xvalid1,bad,axis=1)

xtrain1bis = standardize(xtrain1)
xvalid1bis = standardize(xvalid1)

#### Data preprocessing : perform a pca on xtrain
ratio_pca=0.9

# finding the projector
U, k = my_pca(xtrain1bis, ratio_pca)
print("shape of the projector : ", U.shape)

# projecting the data 
xtrain2=np.dot(xtrain1bis,U)
xvalid2=np.dot(xvalid1bis,U)
print(" shape of the reduced training set : ", xtrain2.shape)
print(" shape of the reduced validation set : ", xvalid2.shape )

### Previous method
#then adding a one before each sample (to enable a constant in linear regression)
#xtrain=np.ones((xtrain2.shape[0],xtrain2.shape[1]+1))
#xvalid=np.ones((xvalid2.shape[0],xvalid2.shape[1]+1))
#xtrain[:,1:]=xtrain2
#xvalid[:,1:]=xvalid2

random selection of samples : done
number of samples :  2000
splitting of the data : done 
number of training samples : 1400
eigenvalues sorted by decreasing order :  [  1.74509274e+04   3.38191456e+03   3.17721928e+03   2.71010066e+03
   2.20280970e+03   2.08454673e+03   1.69292971e+03   1.66586474e+03
   1.43720273e+03   1.27427538e+03   1.08085541e+03   9.04385047e+02
   6.71074533e+02   6.34516357e+02   5.12935208e+02   3.67946531e+02
   2.05115345e+02   1.55621850e+02   1.30380258e+02   1.15730863e+02
   9.05356596e+01   5.20406172e+01   8.68731583e-01   1.72961320e-01
   1.03132600e-02   8.63025968e-03   6.39743888e-03   4.04836181e-03
   3.79779482e-04   1.85291154e-08]
 debugging : check the values :
k :  11
partial sum of eigenvalues :  38158.646269
total sum of eigenvalues :  42000.0
ratio :  0.908539196881
shape of the projector :  (30, 11)
 shape of the reduced training set :  (1400, 11)
 shape of the reduced validation set :  (600, 11)


## Machine Learning : test of the basic functions

In [49]:
### Training of the model : test of least_squares_GD
#weights = train_data(xtrain,ytrain,n_regression=1,gamma=0.0000001,max_iters=500)

### Training of the model : test of least_squares (analytical)
#weights = train_data(xtrain, ytrain, n_regression=3)

### Training of the model : test of least_squares_SGD
#weights = train_data(xtrain, ytrain, n_regression=2, gamma=10e-13, max_iters=500)

### Training of the model : test of ridge_regression
#weights = train_data(xtrain, ytrain, n_regression=4,lambd=100)

#print("Training of the model : done")

In [50]:
### Measure of the MSE for the trained model, over the validation set  : 

#MSE=compute_loss(yvalid,xvalid,weights) 

#print ( "MSE computed, value : ", MSE)

## Machine Learning :  Comparison of polynomial ridge regressions


In [84]:
# First comparison : for the given training and validation set : choice of the best lambd_ for the ridge regression
lambdas = np.logspace(-3, 3, 500) 
degree=7

#setting references
degree_ref=0
w_ref = 0
lambd_ref=0
score_ref = float("inf")
#loop over the degrees
for t in range(1,degree):
    
    #pre-processing the data according to the degree
    xtrain_rr=build_poly(xtrain2,t)
    xvalid_rr=build_poly(xvalid2,t)
    
    #loop over the lambdas
    for lambd_ in lambdas:
        
        #training the model for the ridge regression given lambda
        weights = train_data(xtrain_rr, ytrain, n_regression=4,lambd=lambd_)

        #computing its score
        #mse = compute_loss(yvalid, xvalid, weights)
        yvalid_pred=predict_labels(weights, xvalid_rr)
        score=0
        for i in range(yvalid.shape[0]):
            if (yvalid_pred[i]!=yvalid[i]):
                score=score+1

        #if its score is better than the previous one we keep the (weights, lambda couple)
        #if mse<loss_ref:
        if score<score_ref:
            w_ref=weights
            #loss_ref=mse
            score_ref=score
            lambd_ref=lambd_
            degree_ref=t
        # printing the result for this iteration    
        print("degree : ", str(t),  ", study of the parameter :", str(lambd_), " done. error associated : ", str(score_ref/yvalid.shape[0]) )



degree :  1 , study of the parameter : 0.001  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00102807322383  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00105693455356  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00108660611385  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.0011171106505  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00114847154784  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00118071284667  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00121385926269  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00124793620547  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00128296979789  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.0013189868962  done. error associated :  0.2575
degree :  1 , study of 

KeyboardInterrupt: 

In [52]:
#saving the best parameters
degree_best_rr=degree_ref
weights_rr=w_ref
err_rr=score_ref/yvalid.shape[0]
#loss=loss_ref
print("comparison of the ridge regression parameters : done")
print("best parameter : " , str(lambd_ref))
print("best degree : ", str(degree_best_rr))
print(" associated error : " , str(err_rr))
print("associated weight vector" , str(weights_rr))

comparison of the ridge regression parameters : done
best parameter :  0.001
best degree :  5
 associated error :  0.22866666666666666
associated weight vector [  2.35161423e-01  -7.37255162e-02  -2.19526178e-02  -1.71496139e-03
  -1.22167909e-04  -6.46020866e-06  -4.93386294e-02   1.19831060e-02
   4.75724328e-03  -1.57938714e-03   1.07687219e-04   1.59238577e-01
   6.79495166e-03  -9.12231295e-03  -9.22819418e-04  -1.66814609e-05
   8.50631594e-02  -3.13925588e-02  -5.53305068e-03   7.53221638e-04
  -1.14734441e-05   2.73014994e-01  -4.41516070e-02  -9.85846814e-03
   2.59158818e-03  -1.34037781e-04  -1.38786705e-02  -4.34303918e-02
   1.19731325e-02  -6.46472840e-04  -2.13771537e-03   7.33424531e-03
  -5.68682239e-02  -3.50855113e-03   1.28068498e-02   5.68502941e-04
  -7.72956536e-02  -4.23204258e-02   8.76153442e-04   4.98606287e-04
   1.06304882e-05   2.93870789e-02   1.74105122e-02  -2.34199995e-02
  -8.56325812e-03   6.39222171e-03   1.82196649e-01   1.93329279e-02
  -1.0914187

## Training of the logistic regression

In [117]:
"""
# data already preprocessed. 
#need to add a one at the beginning of each sample : 
xtrain_log = np.c_[np.ones((xtrain2.shape[0], 1)), xtrain2]
xvalid_log=np.c_[np.ones((xvalid2.shape[0], 1)), xvalid2]
# still, we need to preprocess the labels
ytrain_log = pre_process_logistic_training_labels(ytrain)

# training of the model
loss_log, w_log = logistic_regression(ytrain_log, xtrain_log, gamma = 1e-6, max_iter = 50000, threshold = 1e-8)
print("final loss : ", loss_log, ", final weight : ", w_log)
"""
# prediction of the validation labels 
yvalid_log_pred = predict_label_logistic_regression(xvalid_log, w_log)
# post processing (0,1) --> (-1,1)
yvalid_log_pred2 = post_process_logistic_predicted_labels(yvalid_log_pred)
# measure of the error commited
err_log = measure_accuracy(yvalid, yvalid_log_pred2)
print("fraction of error commited : ", err_log)

fraction of error commited :  0.27666666666666667


## Selection of the best model

In [116]:
# for now we just keep the parameters of ridge regression 
# later : comparison of the respective best errors on the validation set 
print (w_log)
print (yvalid_log_pred2)

[[-0.94465306]
 [-0.09113392]
 [-0.58916444]
 [-0.14574351]
 [ 0.2494315 ]
 [ 0.09496332]
 [ 0.34294637]
 [ 0.26550809]
 [-0.05881768]
 [ 0.04542093]
 [ 0.47044807]
 [ 0.27005107]]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.

## Generate predictions and save ouput in csv format for submission:

In [41]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [42]:
# output path 
OUTPUT_PATH = '../results/result4.csv' 

# preprocess the submission data for the ridge regression: 
# 1) standardize
xtest = standardize(tX_test)
# 2) project 
xtest2 = np.dot(xtest, U)
# 3) build polynomial basis
xtest_rr =build_poly(xtest2, degree_best)

# preprocess the submission data for the logistic regression


In [43]:
# predict the labels and save the prediction in a csv file 

# for now : just predict according to the best ridge polynomial regression 
y_pred = predict_labels(weights_rr, xtest_rr)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)